In [1]:
from finlab.data import Data
data = Data()
# 先拿出計算市值需要用到的財務數據

股本 = data.get('股本合計', 1)
price = data.get('收盤價', 100)
# ----------- #
# 找出當天股價  #
# ----------- #

# 新版（較簡單）：將頻率為「天」的股價，reindex 成每「季」
# method='fill'是說，假如公佈財報那天沒有股價，則以時間往前最接近的那天為主。

#當天股價 = price.reindex(股本.index, method='ffill').iloc[-1]
當天股價 =price.iloc[-1]

# 原本（較難）：當天股價 = price.loc[:股本.index[0]].iloc[-1]

# ----------- #
# 找出當天股本  #
# ----------- #

當天股本 = 股本.iloc[-1]

# -----------  #
# 計算市值      #
# -----------  #

# 市值 = 總股數 * 股價
#     = （股本 * 1000） / 10 * 股價

市值 = 當天股本 * 1000 / 10 * 當天股價
市值['1101']

165337820460.0

In [2]:
import pandas as pd

# 將每季累計的財務數據，轉換成單季
def toSeasonal(df):
    season4 = df[df.index.month == 3]
    season1 = df[df.index.month == 5]
    season2 = df[df.index.month == 8]
    season3 = df[df.index.month == 11]

    season1.index = season1.index.year
    season2.index = season2.index.year
    season3.index = season3.index.year
    season4.index = season4.index.year - 1

    newseason1 = season1
    newseason2 = season2 - season1.reindex_like(season2)
    newseason3 = season3 - season2.reindex_like(season3)
    newseason4 = season4 - season3.reindex_like(season4)

    newseason1.index = pd.to_datetime(newseason1.index.astype(str) + '-05-15')
    newseason2.index = pd.to_datetime(newseason2.index.astype(str) + '-08-14')
    newseason3.index = pd.to_datetime(newseason3.index.astype(str) + '-11-14')
    newseason4.index = pd.to_datetime((newseason4.index + 1).astype(str) + '-03-31')

    return newseason1.append(newseason2).append(newseason3).append(newseason4).sort_index()
    
# 計算自由現金流 = 營業活動之淨現金流入 - 投資活動之淨現金流出
投資現金流 = toSeasonal(data.get('投資活動之淨現金流入（流出）', 8))
營業現金流 = toSeasonal(data.get('營業活動之淨現金流入（流出）', 8))
自由現金流 = (投資現金流 + 營業現金流).iloc[-4:].sum()
自由現金流.describe()

稅後淨利 = data.get('本期淨利（淨損）', 1)

# 修正：因為有些股東權益的名稱叫作「權益總計」有些叫作「權益總額」，所以要先將這兩個dataframe合併起來喔！
權益總計 = data.get('權益總計', 1)
權益總額 = data.get('權益總額', 1)

# 把它們合併起來（將「權益總計」為NaN的部分填上「權益總額」）
權益總計.fillna(權益總額, inplace=True)

股東權益報酬率 = 稅後淨利.iloc[-1] / 權益總計.iloc[-1]
股東權益報酬率.describe()


營業利益 = data.get('營業利益（損失）', 5)
營業利益成長率 = (營業利益.iloc[-1] / 營業利益.iloc[-5] - 1) * 100
營業利益成長率.describe()

%matplotlib inline
當月營收 = data.get('當月營收', 4) * 1000
當季營收 = 當月營收.iloc[-4:].sum() # 取近4個月營收總和，當作一季的月營收（4也可以改變）
市值營收比 = 市值 / 當季營收
市值營收比.describe()

count    903.000000
mean            inf
std             NaN
min        0.103960
25%        1.715987
50%        2.936388
75%        5.472453
max             inf
dtype: float64

In [3]:
condition1 = (市值 < 10000000000)
condition2 = 自由現金流 > 0
condition3 = 股東權益報酬率 > 0
condition4 = 營業利益成長率 > 0
condition5 = 市值營收比 < 5

In [4]:
# current
流動資產 = data.get('流動資產合計',5)
流動負債 = data.get('流動負債合計',5)
current_ratio = 流動資產/流動負債
# liability
long_liability = data.get('非流動負債合計',5)
long_liability

#capital
stocks = data.get('普通股股本',5)
#inventory
control_season = 2
inventory = data.get('存貨合計',6)
asset = data.get('資產總計',6)
no_turnover = inventory/asset
turnover_ratio = 1 - no_turnover
turnover_growth_rate = (turnover_ratio.iloc[-control_season] /turnover_ratio.iloc[-control_season-4])

目前擁有的財務數據：
存貨周轉成長率
流動比率
市值營收比
營業利益成長率
股東權益報酬率
稅後淨利
自由現金流

In [5]:
c1 = (市值 < 10000000000)
c2 = 自由現金流 > 0
c3 = 股東權益報酬率 > 0.06
c4 = 營業利益成長率 > 0.1
c5 = 市值營收比 < 5
selects = c1 & c2 & c3 &c4 & c5
selects[selects]

stock_id
1413    True
1441    True
1558    True
2338    True
2342    True
2397    True
2464    True
2614    True
3090    True
3167    True
4536    True
4764    True
4919    True
4999    True
5525    True
6206    True
8996    True
dtype: bool

In [6]:
selects[selects].index

Index(['1413', '1441', '1558', '2338', '2342', '2397', '2464', '2614', '3090',
       '3167', '4536', '4764', '4919', '4999', '5525', '6206', '8996'],
      dtype='object', name='stock_id')